In [1]:
import pandas as pd
import csv
import numpy as np

vote_df = pd.read_csv('US_County_Level_Presidential_Results_12-16.csv')  #Presidential Results 2012, 2016
c2010_df = pd.read_csv('county_facts.csv') #County population size and areas from 2010 census

In [2]:
vote_df.head(2)

,fips,votes_dem_2016,votes_gop_2016,total_votes_2016,per_dem_2016,per_gop_2016,raw_diff_2016,diff_2016,state_abbr,county_name,total_votes_2012,votes_dem_2012,votes_gop_2012,per_dem_2012,per_gop_2012,raw_diff_2012,diff_2012
0,1001,5908,18110,24661,0.239569,0.734358,12202,-0.494789,AL,Autauga County,23909,6354,17366,0.265758,0.726337,11012,-0.460580
1,1003,18409,72780,94090,0.195653,0.773515,54371,-0.577862,AL,Baldwin County,84988,18329,65772,0.215666,0.773897,47443,-0.558232


In [3]:
vote_df.describe()

,fips,votes_dem_2016,votes_gop_2016,total_votes_2016,per_dem_2016,per_gop_2016,raw_diff_2016,diff_2016,total_votes_2012,votes_dem_2012,votes_gop_2012,per_dem_2012,per_gop_2012,raw_diff_2012,diff_2012
count,3112.000000,3.112000e+03,3112.000000,3.112000e+03,3112.000000,3112.000000,3.112000e+03,3112.000000,3.112000e+03,3.112000e+03,3112.000000,3112.000000,3112.000000,3112.000000,3112.000000
mean,30651.700514,2.001874e+04,19600.109576,4.167676e+04,0.317070,0.636152,1.162099e+04,-0.319082,3.950556e+04,1.999004e+04,18890.392352,0.384943,0.597833,9565.503535,-0.212890
std,14984.654885,7.190185e+04,40362.196846,1.132173e+05,0.153578,0.156499,4.270328e+04,0.308811,1.056297e+05,6.599190e+04,41731.449644,0.148024,0.147614,36023.739788,0.295500
min,1001.000000,4.000000e+00,57.000000,6.400000e+01,0.031447,0.041221,2.000000e+00,-0.916364,6.400000e+01,5.000000e+00,54.000000,0.034483,0.060064,1.000000,-0.924138
25%,19038.500000,1.166000e+03,3206.000000,4.820500e+03,0.204759,0.549478,1.599750e+03,-0.546899,4.765750e+03,1.555000e+03,2889.750000,0.278024,0.507527,1047.750000,-0.425556
50%,29208.000000,3.153000e+03,7164.500000,1.094700e+04,0.284739,0.667431,3.746000e+03,-0.382174,1.073250e+04,3.948500e+03,6393.500000,0.371711,0.609411,2565.500000,-0.238108
75%,46005.500000,9.599750e+03,17427.250000,2.879650e+04,0.399610,0.751471,8.510000e+03,-0.149008,2.762825e+04,1.110025e+04,15948.750000,0.475297,0.703178,6473.000000,-0.032239
max,56045.000000,1.893770e+06,620285.000000,2.652072e+06,0.928466,0.952727,1.273485e+06,0.887245,2.427869e+06,1.672164e+06,699600.000000,0.933546,0.958621,972564.000000,0.873482


In [4]:
# Create 2012 and 2016 data sets for modeling
df_2012 = vote_df[['fips', 'county_name', 'diff_2012']]
df_2012 = df_2012.set_index('fips')
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = vote_df[['fips', 'county_name', 'diff_2016']]
df_2016 = df_2016.set_index('fips')
print(df_2016.shape)
#print(df_2016.head(2))

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
df_2016.head(2)

(3112, 2)
(3112, 2)
0
0


,county_name,diff_2016
fips,,
1001,Autauga County,-0.494789
1003,Baldwin County,-0.577862


In [5]:
print(df_2016['diff_2016'].describe())

count    3112.000000
mean       -0.319082
std         0.308811
min        -0.916364
25%        -0.546899
50%        -0.382174
75%        -0.149008
max         0.887245
Name: diff_2016, dtype: float64


In [6]:
c2010_df = c2010_df.set_index('fips')
not_included_fips = list(set(c2010_df.area_name) -  set(df_2012.county_name)) #the dropped fips are state totals 
c2010_df  = c2010_df.drop(columns = ['area_name'])

print(c2010_df.shape)
c2010_df.head(2)

(3194, 3)


,state_abbreviation,2010_population,land_area
fips,,,
1000,NaN,4780127,50645.33
1001,AL,54571,594.44


In [7]:
df_2012 = df_2012.join(c2010_df)
df_2012 = df_2012[['state_abbreviation', 'county_name', 'diff_2012', '2010_population', 'land_area']]
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(c2010_df)
df_2016 = df_2016[['state_abbreviation', 'county_name', 'diff_2016', '2010_population', 'land_area']]
print(df_2016.shape)
#print(df_2016.head(2))

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3112, 5)
(3112, 5)
0
0
count    3112.000000
mean       -0.319082
std         0.308811
min        -0.916364
25%        -0.546899
50%        -0.382174
75%        -0.149008
max         0.887245
Name: diff_2016, dtype: float64


In [8]:
pop_11 = pd.read_csv('SEX_AGE_RACE_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
pop_11 = pop_11.drop(columns = ['county_name'])
pop_11 = pop_11.set_index('fips')
pop_15 = pd.read_csv('SEX_AGE_RACE_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
pop_15 = pop_15.drop(columns = ['county_name'])
pop_15 = pop_15.set_index('fips')
df_2012 = df_2012.join(pop_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(pop_15)
print(df_2016.shape)
#print(df_2016.head(2))

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

(3112, 18)
(3112, 18)
0
1


In [9]:
df_2016[df_2016.isna().any(axis=1)]  #county changed names and fips in 2015, removing it
df_2016 = df_2016[df_2016.index != 46113]
df_2012 = df_2012[df_2012.index != 46113]
print(df_2016['diff_2016'].describe())

count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [10]:
print(df_2012.shape)
print(df_2016.shape)
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
df_2016.head(2)

(3111, 18)
(3111, 18)
0
0


,state_abbreviation,county_name,diff_2016,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,perc_2races,perc_white,perc_black,perc_native,perc_asian,perc_ap,perc_hisp,perc_white_not_hisp
fips,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,-0.494789,54571,594.44,55221.0,51.6,5.9,74.6,13.5,1.6,77.4,18.7,0.4,1.0,0.0,2.6,75.8
1003,AL,Baldwin County,-0.577862,182265,1589.78,195121.0,51.2,5.4,77.6,18.1,1.7,86.4,9.6,0.6,0.7,0.0,4.5,83.1


In [11]:
vets_11 = pd.read_csv('Veterans_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
vets_11 = vets_11.drop(columns = ['county_name'])
vets_11 = vets_11.set_index('fips')
vets_15 = pd.read_csv('Veterans_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
vets_15 = vets_15.drop(columns = ['county_name'])
vets_15 = vets_15.set_index('fips')
df_2012 = df_2012.join(vets_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(vets_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 19)
(3111, 19)
0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [12]:
f_11 = pd.read_csv('Foreign_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
f_11 = f_11.drop(columns = ['county_name'])
f_11 = f_11.set_index('fips')
f_15 = pd.read_csv('Foreign_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
f_15 = f_15.drop(columns = ['county_name'])
f_15 = f_15.set_index('fips')
df_2012 = df_2012.join(f_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(f_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

(3111, 20)
(3111, 20)
0
0


In [13]:
h_11 = pd.read_csv('Housing_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
h_11 = h_11.drop(columns = ['county_name'])
h_11 = h_11.set_index('fips')
h_15 = pd.read_csv('Housing_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
h_15 = h_15.drop(columns = ['county_name'])
h_15 = h_15.set_index('fips')
df_2012 = df_2012.join(h_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(h_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 26)
(3111, 26)
0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [14]:
hh_11 = pd.read_csv('Households_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
hh_11 = hh_11.drop(columns = ['county_name'])
hh_11 = hh_11.set_index('fips')
hh_15 = pd.read_csv('Households_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
hh_15 = hh_15.drop(columns = ['county_name'])
hh_15 = hh_15.set_index('fips')
df_2012 = df_2012.join(hh_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(hh_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

(3111, 28)
(3111, 28)
0
0


In [15]:
df_2012.head(2)
print(df_2016['diff_2016'].describe())

count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [16]:
oy_11 = pd.read_csv('1year_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
oy_11 = oy_11.drop(columns = ['county_name'])
oy_11 = oy_11.set_index('fips')
oy_15 = pd.read_csv('1year_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
oy_15 = oy_15.drop(columns = ['county_name'])
oy_15 = oy_15.set_index('fips')
df_2012 = df_2012.join(oy_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(oy_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

(3111, 29)
(3111, 29)
0
0


In [17]:
fl_11 = pd.read_csv('Language_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
fl_11 = fl_11.drop(columns = ['county_name'])
fl_11 = fl_11.set_index('fips')
fl_15 = pd.read_csv('Language_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
fl_15 = fl_15.drop(columns = ['county_name'])
fl_15 = fl_15.set_index('fips')
df_2012 = df_2012.join(fl_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(fl_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 30)
(3111, 30)
0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [18]:
ed_11 = pd.read_csv('Education_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
ed_11 = ed_11.drop(columns = ['county_name'])
ed_11 = ed_11.set_index('fips')
ed_15 = pd.read_csv('Education_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
ed_15 = ed_15.drop(columns = ['county_name'])
ed_15 = ed_15.set_index('fips')
df_2012 = df_2012.join(ed_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(ed_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

(3111, 32)
(3111, 32)
0
0


In [19]:
emp_11 = pd.read_csv('Employment_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
emp_11 = emp_11.drop(columns = ['county_name'])
emp_11 = emp_11.set_index('fips')
emp_15 = pd.read_csv('Employment_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
emp_15 = emp_15.drop(columns = ['county_name'])
emp_15 = emp_15.set_index('fips')
df_2012 = df_2012.join(emp_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(emp_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 34)
(3111, 34)
0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [20]:
df_2016.head(2)

,state_abbreviation,county_name,diff_2016,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,monthly_costs_nm,median_rent,households,avg_hh_size,perc_1yr_res,perc_f_lang,perc_hs,perc_bc,perc_emp,perc_f_emp_2065
fips,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,-0.494789,54571,594.44,55221.0,51.6,5.9,74.6,13.5,...,354,883,20396,2.68,84.6,3.3,87.2,23.2,61.7,68.8
1003,AL,Baldwin County,-0.577862,182265,1589.78,195121.0,51.2,5.4,77.6,18.1,...,365,879,74104,2.60,85.4,6.0,89.5,29.0,59.6,68.9


In [21]:
ind_12 = pd.read_csv('Industry_2012.csv', encoding="ISO-8859-1")
ind_12.columns = ['state_abbreviation', 'industry', 'type', 'value_1k', 'num_nonemployers']
ind_12 = ind_12[ind_12.industry != 'Other services (except public administration)']
ind_12 = ind_12[ind_12.industry != 'Mining, quarrying, and oil and gas extraction']
ind_12 = ind_12[ind_12.industry != 'Utilities']
ind_12 = ind_12[ind_12.industry != 'Construction']
ind_12 = ind_12[ind_12.industry != 'Transportation and warehousing(104)']
ind_12 = ind_12[ind_12.industry != 'Information']
ind_12 = ind_12[ind_12.industry != 'Finance and insurance']
ind_12 = ind_12[ind_12.industry != 'Real estate and rental and leasing']
ind_12 = ind_12[ind_12.industry != 'Professional, scientific, and technical service']
ind_12 = ind_12[ind_12.industry != 'Management of companies and enterprises']
ind_12 = ind_12[ind_12.industry != 'Administrative and support and waste management and remediation services']
ind_12 = ind_12[ind_12.industry != 'Educational services']
ind_12 = ind_12[ind_12.industry != 'Arts, entertainment, and recreation']
ind_12 = ind_12[ind_12.industry != 'Professional, scientific, and technical services']
ind_12 = ind_12[ind_12.type != "Merchant wholesalers, except manufacturers' sales branches and offices"]
ind_12 = ind_12[ind_12.type != "Manufacturers' sales branches and offices"]
ind_12 = ind_12[ind_12.type != "Establishments subject to federal income tax"]
ind_12 = ind_12[ind_12.type != "Establishments exempt from federal income tax"]
ind_12  = ind_12.drop(columns = ['type'])
ind_12 = ind_12.replace({'value_1k': 'D'}, -100)
ind_12['value_1k'] = ind_12['value_1k'].str.replace(r'\D', '').astype(float)
ind_12 = ind_12.replace({'value_1k': -100}, ind_12.value_1k.mean())
#ind_12.head(5)
print(df_2016['diff_2016'].describe())

count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [22]:
num_ne = ind_12[['state_abbreviation', 'num_nonemployers']]
num_ne = num_ne.astype({'num_nonemployers': 'int64'})
num_ne = num_ne.groupby('state_abbreviation').sum()

man_value = (ind_12.loc[ind_12['industry'] == 'Manufacturing']).drop(columns = ['industry', 'num_nonemployers'])
man_value = man_value.rename(columns = {'value_1k':'man_value_1k'})
wholesale_value = (ind_12.loc[ind_12['industry'] == 'Wholesale trade']).drop(columns = ['industry', 'num_nonemployers'])
wholesale_value = wholesale_value.rename(columns = {'value_1k':'wholesale_value_1k'})
retail_value = (ind_12.loc[ind_12['industry'] == 'Retail trade']).drop(columns = ['industry', 'num_nonemployers'])
retail_value = retail_value.rename(columns = {'value_1k':'retail_value_1k'})
hc_value = (ind_12.loc[ind_12['industry'] == 'Health care and social assistance']).drop(columns = ['industry', 'num_nonemployers'])
hc_value = hc_value.rename(columns = {'value_1k':'hc_value_1k'})
food_value = (ind_12.loc[ind_12['industry'] == 'Accommodation and food services']).drop(columns = ['industry', 'num_nonemployers'])
food_value = food_value.rename(columns = {'value_1k':'food_value_1k'})

num_ne['man_value_1k'] = list(man_value['man_value_1k'])
num_ne['wholesale_value_1k'] = list(wholesale_value['wholesale_value_1k'])
num_ne['retail_value_1k'] = list(retail_value['retail_value_1k'])
num_ne['hc_value_1k'] = list(hc_value['hc_value_1k'])
num_ne['food_value_1k'] = list(food_value['food_value_1k'])

df_2012 = df_2012.join(num_ne, on='state_abbreviation')
print(df_2012.shape)
df_2016 = df_2016.join(num_ne, on='state_abbreviation')
print(df_2016.shape)
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

(3111, 40)
(3111, 40)
4
4


In [23]:
inc_11 = pd.read_csv('Median_Income_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
inc_11 = inc_11.drop(columns = ['county_name'])
inc_11 = inc_11.set_index('fips')
inc_15 = pd.read_csv('Median_Income_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
inc_15 = inc_15.drop(columns = ['county_name'])
inc_15 = inc_15.set_index('fips')
df_2012 = df_2012.join(inc_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(inc_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 41)
(3111, 41)
4
4
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [24]:
pcinc_11 = pd.read_csv('PerCapitaIncome_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
pcinc_11 = pcinc_11.drop(columns = ['county_name'])
pcinc_11 = pcinc_11.set_index('fips')
pcinc_15 = pd.read_csv('PerCapitaIncome2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
pcinc_15 = pcinc_15.drop(columns = ['county_name'])
pcinc_15 = pcinc_15.set_index('fips')
df_2012 = df_2012.join(pcinc_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(pcinc_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 42)
(3111, 42)
4
4
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [25]:
pov_11 = pd.read_csv('Poverty_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
pov_11 = pov_11.drop(columns = ['county_name'])
pov_11 = pov_11.set_index('fips')
pov_15 = pd.read_csv('Poverty_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
pov_15 = pov_15.drop(columns = ['county_name'])
pov_15 = pov_15.set_index('fips')
df_2012 = df_2012.join(pov_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(pov_15)
print(df_2016.shape)
#print(df_2016.head(2))
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 43)
(3111, 43)
4
4
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [26]:
bus_11 = pd.read_csv('Businesses_2011.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2011
bus_11 = bus_11[bus_11.industry == 'Total for all sectors']
bus_11 = bus_11.drop(columns = ['county_name', 'industry'])
bus_11 = bus_11.set_index('fips')
print(bus_11.num_employees[bus_11.num_employees > '99999999999'].count())
print(bus_11.annual_payroll[bus_11.annual_payroll > '99999999999'].count())
bus_11 = bus_11.replace(['a', 'b', 'c', 'd', 'e', 'f','g', 'h','i','j','k','l', 'D'], -100)
bus_11 = bus_11.astype(int)

bus_11 = bus_11.replace({'num_employees': -100}, bus_11.num_employees.mean())
bus_11 = bus_11.replace({'annual_payroll': -100}, bus_11.annual_payroll.mean())

bus_15 = pd.read_csv('Businesses_2015.csv', encoding="ISO-8859-1") #Population, Age, Race, Sex Demographics 2015
bus_15 = bus_15[bus_15.industry == 'Total for all sectors']
bus_15 = bus_15.drop(columns = ['county_name', 'industry'])
bus_15 = bus_15.set_index('fips')
print(bus_15.num_employees[bus_15.num_employees > '99999999999'].count())
print(bus_15.annual_payroll[bus_15.annual_payroll > '99999999999'].count())
bus_15 = bus_15.replace(['a', 'b', 'c', 'd', 'e', 'f','g', 'h','i','j','k','l', 'D'], -100)
bus_15 = bus_15.astype(int)
bus_15 = bus_15.replace({'num_employees': -100}, bus_15.num_employees.mean())
bus_15 = bus_15.replace({'annual_payroll': -100}, bus_15.annual_payroll.mean())

df_2012 = df_2012.join(bus_11)
print(df_2012.shape)
#print(df_2012.head(2))

df_2016 = df_2016.join(bus_15)
print(df_2016.shape)
#print(df_2016.head(2))

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

50
28
1
1
(3111, 46)
(3111, 46)
6
5
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [27]:
df_2012.head(2)

,state_abbreviation,county_name,diff_2012,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,wholesale_value_1k,retail_value_1k,hc_value_1k,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll
fips,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,-0.460580,54571,594.44,53944,51.5,6.6,72.7,11.7,...,75844516.0,58564965.0,26039632.0,7576462.0,53899,25035,10.9,835.0,10290.0,276674.0
1003,AL,Baldwin County,-0.558232,182265,1589.78,179523,51.2,6.2,76.8,16.7,...,75844516.0,58564965.0,26039632.0,7576462.0,51321,27217,12.5,4624.0,51386.0,1516145.0


In [28]:
df_2012=df_2012.fillna(df_2012.annual_payroll.mean())
df_2012=df_2012.fillna(df_2012.num_establishments.mean())
df_2012=df_2012.fillna(df_2012.num_employees.mean())

df_2016=df_2016.fillna(df_2016.annual_payroll.mean())
df_2016=df_2016.fillna(df_2016.num_establishments.mean())
df_2016=df_2016.fillna(df_2016.num_employees.mean())

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [29]:
sbo = pd.read_csv('SBO_2012.csv', encoding="ISO-8859-1")
sbo = sbo.drop(columns = ['county_name'])
sbo_tot = sbo[sbo.veteran == 'All firms']
sbo_tot = sbo_tot[sbo_tot.gender == 'All firms']
sbo_tot = sbo_tot.drop(columns = ['veteran', 'gender', 'num_firms'])
sbo_tot = sbo_tot.set_index('fips')
#sbo_tot = sbo_tot.rename(columns = {'num_firms':'tot_num_firms'})
df_2012 = df_2012.join(sbo_tot)
df_2016 = df_2016.join(sbo_tot)

print(df_2012.shape)
print(df_2016.shape)

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 46)
(3111, 46)
0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [30]:
#df_2012=df_2012.fillna(df_2012.tot_num_firms.mean())
#df_2016=df_2016.fillna(df_2016.tot_num_firms.mean())
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [31]:
sbo_vet = sbo[sbo.veteran == 'Veteran-owned']
sbo_vet = sbo_vet[sbo_vet.gender == 'All firms']
sbo_vet = sbo_vet.drop(columns = ['veteran', 'gender'])
sbo_vet = sbo_vet.set_index('fips')
sbo_vet = sbo_vet.rename(columns = {'num_firms':'vet_num_firms'})
df_2012 = df_2012.join(sbo_vet)
df_2016 = df_2016.join(sbo_vet)

print(df_2012.shape)
print(df_2016.shape)

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 47)
(3111, 47)
228
228
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [32]:
df_2012=df_2012.fillna(df_2012.vet_num_firms.mean())
df_2016=df_2016.fillna(df_2016.vet_num_firms.mean())
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [33]:
sbo_f = sbo[sbo.veteran == 'All firms']
sbo_f = sbo_f[sbo_f.gender == 'Female-owned']
sbo_f = sbo_f.drop(columns = ['veteran', 'gender'])
sbo_f = sbo_f.set_index('fips')
sbo_f = sbo_f.rename(columns = {'num_firms':'fem_num_firms'})
df_2012 = df_2012.join(sbo_f)
df_2016 = df_2016.join(sbo_f)


print(df_2012.shape)
print(df_2016.shape)

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))

print(df_2016['diff_2016'].describe())

(3111, 48)
(3111, 48)
49
49
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [34]:
df_2012=df_2012.fillna(df_2012.fem_num_firms.mean())
df_2016=df_2016.fillna(df_2016.fem_num_firms.mean())
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [35]:
sbo_m = pd.read_csv('SBO_Race_2012.csv', encoding="ISO-8859-1")
sbo_m = sbo_m.drop(columns = ['county_name'])
sbo_m = sbo_m.set_index('fips')
df_2012 = df_2012.join(sbo_m)
df_2016 = df_2016.join(sbo_m)

print(df_2012.shape)
print(df_2016.shape)

print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

(3111, 49)
(3111, 49)
564
564
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [36]:
df_2012=df_2012.fillna(df_2012.min_num_firms.mean())
df_2016=df_2016.fillna(df_2016.min_num_firms.mean())
print(len(df_2012[df_2012.isna().any(axis=1)]))
print(len(df_2016[df_2016.isna().any(axis=1)]))
print(df_2016['diff_2016'].describe())

0
0
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [37]:
print(df_2012.shape)
print(df_2016.shape)
df_2016.head(2)

(3111, 49)
(3111, 49)


,state_abbreviation,county_name,diff_2016,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,-0.494789,54571,594.44,55221.0,51.6,5.9,74.6,13.5,...,7576462.0,51281,24974,12.9,844.0,10454.0,321433.0,285.0,1093.0,616.0
1003,AL,Baldwin County,-0.577862,182265,1589.78,195121.0,51.2,5.4,77.6,18.1,...,7576462.0,50254,27317,13.4,5127.0,58868.0,1887256.0,2364.0,6505.0,2130.0


In [38]:
df_2016.dtypes

state_abbreviation      object
county_name             object
diff_2016              float64
2010_population          int64
land_area              float64
population             float64
perc_female            float64
perc_u5                float64
perc_o18               float64
perc_o65               float64
perc_2races            float64
perc_white             float64
perc_black             float64
perc_native            float64
perc_asian             float64
perc_ap                float64
perc_hisp              float64
perc_white_not_hisp    float64
perc_vets              float64
perc_foreign           float64
housing_units            int64
perc_oo                float64
median_oo_value         object
monthly_costs_m         object
monthly_costs_nm        object
median_rent             object
households               int64
avg_hh_size            float64
perc_1yr_res           float64
perc_f_lang            float64
perc_hs                float64
perc_bc                float64
perc_emp

In [39]:
#df_2016['median_oo_value'] = df_2016['median_oo_value'].str.replace(r'\D', '').astype(float)

df_2016 = df_2016.replace({'median_oo_value': '(X)'}, -100)
df_2016 = df_2016.astype({'median_oo_value': 'float'})
df_2016 = df_2016.replace({'median_oo_value': -100}, df_2016.median_oo_value.mean())



print(df_2016['diff_2016'].describe())


count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [40]:
df_2016 = df_2016.replace({'monthly_costs_m': '(X)'}, -100)
df_2016 = df_2016.astype({'monthly_costs_m': 'float'})
df_2016 = df_2016.replace({'monthly_costs_m': -100}, df_2016.monthly_costs_m.mean())


df_2016 = df_2016.replace({'monthly_costs_nm': '(X)'}, -100)
df_2016 = df_2016.astype({'monthly_costs_nm': 'float'})
df_2016 = df_2016.replace({'monthly_costs_nm': -100}, df_2016.monthly_costs_nm.mean())

print(df_2016['diff_2016'].describe())


count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [41]:
df_2016 = df_2016.replace({'median_rent': '(X)'}, -100)
df_2016 = df_2016.astype({'median_rent': 'float'})
df_2016 = df_2016.replace({'median_rent': -100}, df_2016.median_rent.mean())

print(df_2016['diff_2016'].describe())


count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [42]:
df_2016 = df_2016.replace({'median_income': '(X)'}, -100)
df_2016 = df_2016.astype({'median_income': 'float'})
df_2016 = df_2016.replace({'median_income': -100}, df_2016.median_income.mean())
print(df_2016['diff_2016'].describe())
df_2012['perc_vets'] = df_2012['perc_vets'].str.replace(r'\D', '').astype(float)/100
print(df_2016['diff_2016'].describe())
df_2012['monthly_costs_nm'] = df_2012['monthly_costs_nm'].str.replace(r'\D', '').astype(float)
print(df_2016['diff_2016'].describe())



count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64
count    3111.000000
mean       -0.319436
std         0.308229
min        -0.916364
25%        -0.546912
50%        -0.382228
75%        -0.149127
max         0.887245
Name: diff_2016, dtype: float64


In [43]:
df_2012[df_2012.isna().any(axis=1)]

,state_abbreviation,county_name,diff_2012,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,


In [44]:
df_2016[df_2016.isna().any(axis=1)]

,state_abbreviation,county_name,diff_2016,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,


In [45]:
df_2016.dtypes



state_abbreviation      object
county_name             object
diff_2016              float64
2010_population          int64
land_area              float64
population             float64
perc_female            float64
perc_u5                float64
perc_o18               float64
perc_o65               float64
perc_2races            float64
perc_white             float64
perc_black             float64
perc_native            float64
perc_asian             float64
perc_ap                float64
perc_hisp              float64
perc_white_not_hisp    float64
perc_vets              float64
perc_foreign           float64
housing_units            int64
perc_oo                float64
median_oo_value        float64
monthly_costs_m        float64
monthly_costs_nm       float64
median_rent            float64
households               int64
avg_hh_size            float64
perc_1yr_res           float64
perc_f_lang            float64
perc_hs                float64
perc_bc                float64
perc_emp

In [46]:
df_2012.dtypes

state_abbreviation      object
county_name             object
diff_2012              float64
2010_population          int64
land_area              float64
population               int64
perc_female            float64
perc_u5                float64
perc_o18               float64
perc_o65               float64
perc_2races            float64
perc_white             float64
perc_black             float64
perc_native            float64
perc_asian             float64
perc_ap                float64
perc_hisp              float64
perc_white_not_hisp    float64
perc_vets              float64
perc_foreign           float64
housing_units            int64
perc_oo                float64
median_oo_value          int64
monthly_costs_m          int64
monthly_costs_nm       float64
median_rent              int64
households               int64
avg_hh_size            float64
perc_1yr_res           float64
perc_f_lang            float64
perc_hs                float64
perc_bc                float64
perc_emp

In [47]:
df_2012.head(2)

,state_abbreviation,county_name,diff_2012,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,-0.460580,54571,594.44,53944,51.5,6.6,72.7,11.7,...,7576462.0,53899,25035,10.9,835.0,10290.0,276674.0,285.0,1093.0,616.0
1003,AL,Baldwin County,-0.558232,182265,1589.78,179523,51.2,6.2,76.8,16.7,...,7576462.0,51321,27217,12.5,4624.0,51386.0,1516145.0,2364.0,6505.0,2130.0


In [48]:
df_2016.head(2)

,state_abbreviation,county_name,diff_2016,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,...,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,-0.494789,54571,594.44,55221.0,51.6,5.9,74.6,13.5,...,7576462.0,51281.0,24974,12.9,844.0,10454.0,321433.0,285.0,1093.0,616.0
1003,AL,Baldwin County,-0.577862,182265,1589.78,195121.0,51.2,5.4,77.6,18.1,...,7576462.0,50254.0,27317,13.4,5127.0,58868.0,1887256.0,2364.0,6505.0,2130.0


In [49]:
df_2012.to_csv('df_2012.csv', index = True)
df_2016.to_csv('df_2016.csv', index = True)


In [50]:
df_2016.describe()

,diff_2016,2010_population,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,perc_2races,perc_white,...,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,num_employees,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
count,3111.000000,3.111000e+03,3111.000000,3.111000e+03,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,...,3.111000e+03,3111.000000,3111.000000,3111.000000,3.111000e+03,3.111000e+03,3.111000e+03,3111.000000,3111.000000,3111.000000
mean,-0.319436,9.901452e+04,951.190389,1.015003e+05,49.985310,5.929090,77.290260,17.195468,2.171231,83.856959,...,1.643611e+07,46668.782627,24280.151720,16.721858,3.070587e+03,3.904370e+04,1.934683e+06,881.035727,3226.944807,3120.780526
std,0.308229,3.143772e+05,1303.340446,3.245016e+05,2.358685,1.222148,3.385602,4.369361,1.849955,16.156379,...,1.828814e+07,12118.953775,5763.628558,6.542606,3.566906e+04,1.465610e+05,9.014545e+06,2468.691148,13016.292694,17190.310139
min,-0.916364,8.200000e+01,2.000000,1.170000e+02,26.800000,0.000000,59.700000,3.300000,0.000000,10.200000,...,1.564272e+06,19328.000000,8292.000000,1.400000,3.000000e+00,4.000000e+00,2.960000e+02,25.000000,26.000000,25.000000
25%,-0.546912,1.130900e+04,430.115000,1.120500e+04,49.500000,5.200000,75.500000,14.400000,1.200000,77.550000,...,4.789281e+06,38750.000000,20443.000000,12.000000,2.255000e+02,2.196000e+03,7.503600e+04,117.500000,290.500000,113.000000
50%,-0.382228,2.607600e+04,610.930000,2.601200e+04,50.500000,5.900000,77.300000,16.800000,1.800000,90.200000,...,1.169795e+07,45025.000000,23553.000000,16.000000,5.460000e+02,6.535000e+03,2.341670e+05,275.000000,667.000000,441.000000
75%,-0.149127,6.709650e+04,914.495000,6.795050e+04,51.100000,6.500000,79.300000,19.600000,2.600000,95.500000,...,1.897661e+07,52109.500000,27066.000000,20.300000,1.489000e+03,2.035350e+04,7.958195e+05,881.000000,1873.500000,3120.780526
max,0.887245,9.818664e+06,20056.940000,1.003839e+07,59.400000,14.800000,94.900000,50.900000,28.600000,100.000000,...,9.083037e+07,123453.000000,65600.000000,48.700000,1.934683e+06,4.007163e+06,2.393404e+08,69608.000000,439513.000000,631218.000000
